In [1]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
import pickle
import re
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV
from scipy import signal

#filename queue
filename_queue=glob.glob('./label/label[0-9]*.csv')
#filename_queue=glob.glob('./label/label[6].csv')


In [2]:
training_set=list()
training_set2=list()
test_set=list()
training_label=list()
test_label=list()
index=[1,2,3,11,14,15]
label_count=np.zeros(16)
traininglabel_count=np.zeros(16)
testlabel_count=np.zeros(16)
ldata=list()

for filename in filename_queue:
    with open(filename,newline='') as data:
        
        data_set=list()
        data_set2=list()
        data_label=list()
        reader=csv.reader(data)
        label=re.findall('\d+',filename)
        label=int(label[0])
        label_count[label]+=4
        if label in index:
            continue
        if label not in index:
            print('label : ', label)
            for row in reader:
                row[0:]=[float(i) for i in row[0:]]
                X_acc=list(row[i] for i in range(150))
                Y_acc=list(row[i+150] for i in range(150))
                Z_acc=list(row[i+300] for i in range(150))
                X_gyro=list(row[i+450] for i in range(150))
                Y_gyro=list(row[i+600] for i in range(150))
                Z_gyro=list(row[i+750] for i in range(150))

                window=np.array([X_acc[0:25],Y_acc[0:25],Z_acc[0:25],X_gyro[0:25],Y_gyro[0:25],Z_gyro[0:25],
                                  X_acc[25:50],Y_acc[25:50],Z_acc[25:50],X_gyro[25:50],Y_gyro[25:50],Z_gyro[25:50],
                                  X_acc[50:75],Y_acc[50:75],Z_acc[50:75],X_gyro[50:75],Y_gyro[50:75],Z_gyro[50:75],
                                  X_acc[75:100],Y_acc[75:100],Z_acc[75:100],X_gyro[75:100],Y_gyro[75:100],Z_gyro[75:100],
                                  X_acc[100:125],Y_acc[100:125],Z_acc[100:125],X_gyro[100:125],Y_gyro[100:125],Z_gyro[100:125],
                                  X_acc[125:150],Y_acc[125:150],Z_acc[125:150],X_gyro[125:150],Y_gyro[125:150],Z_gyro[125:150]])

                
                window2=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]]])
               
            
            
                """
                window4=np.array([[X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])
                """
                
                
                
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)
                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)
                
                #Drop outliers1 
                xm=window_stddev[[0,6,12,18,24,30]].mean(axis=0)
                ym=window_stddev[[1,7,13,19,25,31]].mean(axis=0)
                zm=window_stddev[[2,8,14,20,26,32]].mean(axis=0)
                action=np.mean([xm,ym,zm])
                if(action<0.65):
                    label_count[label]-=1
                    continue
                
                window2=window2.reshape(-1)
                #window_RMS=np.roots(np.square(window2[0:6]).mean(axis=-1))
                #if (label_count[label]>=0):
                data_set.append(window_feature)
                data_set2.append(window2)
                data_label.append(label)
            
                #else:
                #    test_set.append(window_feature)
                #    test_label.append(label)
                #    testlabel_count[label]+=1
                
            from sklearn.manifold import TSNE
            from sklearn.decomposition import PCA
            from sklearn.decomposition import MiniBatchDictionaryLearning
            from sklearn.decomposition import IncrementalPCA
            from sklearn.decomposition import TruncatedSVD


            print(np.shape(data_set2))

            data_set2=np.array(data_set2)
            X_embedded2 = MiniBatchDictionaryLearning(n_components=3,n_iter=1000).fit_transform(data_set2)
            #fig = plt.figure()
            #ax2 = fig.add_subplot(111, projection='3d')
            #for x in X_embedded2:
            #    ax2.scatter(x[0],x[1],x[2]) 

            from scipy import stats
            z = np.abs(stats.zscore(X_embedded2))
            #X_embedded2 = X_embedded2[(abs(z) <1.8).all(axis=1)]
            data_set_arr=np.array(data_set)
            data_label_arr=np.array(data_label)
            data_set_arr=data_set_arr[(abs(z) <2).all(axis=1)]
            data_label_arr=data_label_arr[(abs(z) <2).all(axis=1)]

            #print(np.shape(data_set_arr))
            #print(np.shape(data_label_arr))

            for x in data_set_arr:
                training_set.append(x)
            for y in data_label_arr:
                training_label.append(y)
            #fig2 = plt.figure()
            #ax2 = fig2.add_subplot(111, projection='3d')
            #for x in X_embedded2:
            #    ax2.scatter(x[0],x[1],x[2]) 


label :  12
(248, 450)
label :  8
(68, 450)
label :  7
(68, 450)
label :  4
(4, 450)
label :  5
(136, 450)
label :  13
(36, 450)
label :  9
(72, 450)
label :  10
(136, 450)
label :  0
(124, 450)
label :  6
(228, 450)


In [3]:
X=training_set
y=training_label

print(np.shape(X))

(1027, 180)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#X=preprocessing.minmax_scale(training_set)
#X=preprocessing.robust_scale(training_set)
#X=preprocessing.quantile_transform(training_set)

X=training_set
y=training_label
RF=RandomForestClassifier()
param_grid={
 'n_estimators':[50,80,100,120]
   # 'max_features':['auto','sqrt','log2'],
    #'max_depth':[10,12,None]
    #'criterion':['gini','entropy']
}
grid=GridSearchCV(estimator=RF,param_grid=param_grid,cv=10,scoring='accuracy')

In [5]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [6]:
grid.cv_results_

{'mean_fit_time': array([0.31167181, 0.4762229 , 0.51015446, 0.73626378]),
 'std_fit_time': array([0.03497974, 0.02999419, 0.02934838, 0.09634957]),
 'mean_score_time': array([0.00417123, 0.00783632, 0.00787406, 0.01027536]),
 'std_score_time': array([0.00134353, 0.00283589, 0.00200646, 0.0037204 ]),
 'param_n_estimators': masked_array(data=[50, 80, 100, 120],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 50},
  {'n_estimators': 80},
  {'n_estimators': 100},
  {'n_estimators': 120}],
 'split0_test_score': array([0.8317757 , 0.85046729, 0.85981308, 0.85046729]),
 'split1_test_score': array([1.        , 0.97196262, 0.96261682, 0.96261682]),
 'split2_test_score': array([0.91588785, 1.        , 0.99065421, 0.96261682]),
 'split3_test_score': array([0.88785047, 0.89719626, 0.86915888, 0.88785047]),
 'split4_test_score': array([0.92156863, 0.88235294, 0.91176471, 0.87254902]),
 'split5_test_score': array([0.9

In [46]:
RF=RandomForestClassifier(grid.best_params_)

In [87]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
X=preprocessing.quantile_transform(training_set)
ETC=ExtraTreesClassifier()

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (998). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [88]:
param_grid={
 'n_estimators':[100,200,300],
   # 'max_features':['auto','sqrt','log2'],
    'max_depth':[10,14,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=ETC,param_grid=param_grid,cv=10,scoring='accuracy')

In [89]:
grid.fit(X,y)


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None,
                                            criterion='gini', max_depth=None,
                                            max_features='auto',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators='warn', n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 14,

In [90]:
grid.cv_results_

{'mean_fit_time': array([0.12705123, 0.25068548, 0.40250912, 0.13965411, 0.26779549,
        0.39827914, 0.13415592, 0.27339425, 0.4008352 ]),
 'std_fit_time': array([0.00479177, 0.00462072, 0.03329052, 0.01883117, 0.01545729,
        0.01457844, 0.00514715, 0.01448693, 0.01145793]),
 'mean_score_time': array([0.0061923 , 0.01181958, 0.0177851 , 0.00638855, 0.0121907 ,
        0.01795757, 0.0072149 , 0.0122931 , 0.01842859]),
 'std_score_time': array([5.87016561e-05, 1.15543333e-04, 4.95561748e-04, 3.12251785e-04,
        6.53587860e-04, 7.71576453e-04, 1.35826190e-03, 2.91940527e-04,
        1.30989841e-03]),
 'param_max_depth': masked_array(data=[10, 10, 10, 14, 14, 14, None, None, None],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[100, 200, 300, 100, 200, 300, 100, 200, 300],
              mask=[False, False, False, False, False, F

In [91]:
grid.best_params_

{'max_depth': None, 'n_estimators': 200}

In [92]:
ETC=ExtraTreesClassifier(max_depth=None,n_estimators=200).fit(X,y)

In [9]:
#SVM
#prepare the data
#prepare the model
#X=preprocessing.minmax_scale(training_set)
#X=preprocessing.robust_scale(training_set)
X=preprocessing.quantile_transform(training_set)
y=training_label
clf = sklearn.svm.SVC(gamma='scale',tol=0.1,probability=True)
scores=cross_val_score(clf,X,y,cv=10,scoring='accuracy')
print(scores.mean())

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.8847188866875395


In [58]:
param_grid={
 #'kernel':['linear','poly','rbf','sigmoid'], rbf to be best acc
    'C':[10,20]}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

In [10]:
grid.fit(X,y)
grid.cv_results_

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'mean_fit_time': array([0.24235089, 0.3697392 , 0.46288066, 0.55397198]),
 'std_fit_time': array([0.01884969, 0.00530118, 0.00664088, 0.00683818]),
 'mean_score_time': array([0.00353911, 0.00517144, 0.00630219, 0.00735407]),
 'std_score_time': array([0.00034873, 0.00017906, 0.00019981, 0.00010762]),
 'param_n_estimators': masked_array(data=[50, 80, 100, 120],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 50},
  {'n_estimators': 80},
  {'n_estimators': 100},
  {'n_estimators': 120}],
 'split0_test_score': array([0.78504673, 0.85981308, 0.85046729, 0.8317757 ]),
 'split1_test_score': array([0.98130841, 0.93457944, 0.96261682, 0.94392523]),
 'split2_test_score': array([0.92523364, 0.96261682, 0.98130841, 0.99065421]),
 'split3_test_score': array([0.92523364, 0.88785047, 0.88785047, 0.88785047]),
 'split4_test_score': array([0.91176471, 0.88235294, 0.91176471, 0.92156863]),
 'split5_test_score': array([0.9

In [11]:
grid.best_params_

{'n_estimators': 100}

In [12]:
SVC=sklearn.svm.SVC(100,probability=True).fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [13]:
#save the model

from joblib import dump, load
filename='./svm.sav'
pickle.dump(SVC,open(filename,'wb'))

In [35]:
filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')

['rf.joblib']

In [93]:
filename='./ETC.sav'
pickle.dump(ETC,open(filename,'wb'))
dump(ETC,'ETC.joblib')

['ETC.joblib']

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import TruncatedSVD



fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')

labl=9
X=training_set
y=labl
k=0
#X_embedded1 = TSNE(n_components=3).fit_transform(X)
X_embedded1 = PCA(n_components=3).fit_transform(X)
X_embedded2 = MiniBatchDictionaryLearning(n_components=3,n_iter=1000).fit_transform(X)
X_embedded3 = IncrementalPCA(n_components=3).fit_transform(X)
X_embedded4 = TruncatedSVD(n_components=3,n_iter=500).fit_transform(X)
X_embedded5 = FactorAnalysis(n_components=3).fit_transform(X)


   
for x in X_embedded1:
    ax1.scatter(x[0],x[1],x[2]) 
    
#for x in X_embedded3:
#    ax.scatter(x[0],x[1],x[2]) 
    